# lgb model

In [1]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
# # monitor 
# def get_memory_usage():
#     return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
# def sizeof_fmt(num, suffix='B'):
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f%s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f%s%s" % (num, 'Yi', suffix)

## Load data

In [3]:
# change the file path if run on different machines
# FilePath = "/Users/yabindong/Program_Dataset/M5-Forcasting/m5-forecasting-accuracy/"
FilePath = "MainData/"

In [4]:
# define a function to reduce the memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2 #bytes to MB
    
    # the for loop converts int16 --> int8, int32 --> int 16, etc
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[0:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('Memory usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, 100*(start_mem-end_mem)/start_mem))
            
    return df

In [5]:
# read data and reduce memory usage
def ReadData(Path):
    print("Reading files...")
    calendar = pd.read_csv(FilePath+'calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print("calendar df has {} rows and {} columns".format(calendar.shape[0], calendar.shape[1]))
    
    train = pd.read_csv(FilePath+'sales_train_validation.csv')
    train = reduce_mem_usage(train)
    print("train df has {} rows and {} columns".format(train.shape[0], train.shape[1]))
    
    SellPrice = pd.read_csv(FilePath+'sell_prices.csv')
    SellPrice = reduce_mem_usage(SellPrice)
    print("train df has {} rows and {} columns".format(SellPrice.shape[0], SellPrice.shape[1]))
    
    SampleSub = pd.read_csv(FilePath+'sample_submission.csv')
    SampleSub = reduce_mem_usage(SampleSub)
    print("train df has {} rows and {} columns".format(SampleSub.shape[0], SampleSub.shape[1]))
    
    return calendar, train, SellPrice, SampleSub

In [6]:
df_calendar0, df_train0, df_SellPrice0, df_Sample_Submission = ReadData(FilePath)

Reading files...
Memory usage decreased from  0.21 Mb to  0.12 Mb (41.9% reduction)
calendar df has 1969 rows and 14 columns
Memory usage decreased from 446.40 Mb to 95.00 Mb (78.7% reduction)
train df has 30490 rows and 1919 columns
Memory usage decreased from 208.77 Mb to 130.48 Mb (37.5% reduction)
train df has 6841121 rows and 4 columns
Memory usage decreased from 13.49 Mb to  2.09 Mb (84.5% reduction)
train df has 60980 rows and 29 columns


## Transform data table

### Part 1

**Train df**

In [7]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1913         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [8]:
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
# unpivot a table from wide to long
df_grid = pd.melt(df_train0, id_vars = index_columns, var_name = 'd', value_name = TARGET)
print("the shape of df_grid is {}".format(df_grid.shape))

the shape of df_grid is (58327370, 8)


**add test set**

In [9]:
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = df_train0[index_columns].drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])
print("The shape of added test dataframe is {}".format(add_grid.shape)) # 30490*28=853720

The shape of added test dataframe is (853720, 8)


In [10]:
df_grid = pd.concat([df_grid,add_grid])
df_grid = df_grid.reset_index(drop=True)
print("The shape of df_grid is {}".format(df_grid.shape))

The shape of df_grid is (59181090, 8)


In [11]:
# # remove not used df
# del temp_df, add_grid, df_train0

In [12]:
# Reduce memory usage by converting "strings" to categorical and it will not lose info
for col in index_columns:
    df_grid[col] = df_grid[col].astype('category')

**price df**

In [13]:
# Merging by concat to not lose data
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [14]:
# find absence for the item in the store, the first none 0 wm_yr_wk
df_release = df_SellPrice0.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
df_release.columns = ['store_id','item_id','release']

In [15]:
# merge price and train
df_grid = merge_by_concat(df_grid, df_release, ['store_id', 'item_id'])
print("The shape of df_grid is {}".format(df_grid.shape))

The shape of df_grid is (59181090, 9)


In [16]:
# del df_release

**calendar df**

In [17]:
df_grid = merge_by_concat(df_grid, df_calendar0[['wm_yr_wk','d']], ['d'])

In [18]:
# cut off rows by release date
df_grid = df_grid[df_grid['wm_yr_wk']>=df_grid['release']]

In [19]:
df_grid = df_grid.reset_index(drop=True)

In [20]:
df_grid.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12.0,11101,11101
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2.0,11101,11101
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,11101
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,11101
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4.0,11101,11101


In [21]:
# save part 1
df_grid.to_pickle(FilePath+'grid_part_1.pkl')
print("Size of grid_part_1: {}".format(df_grid.shape))

Size of grid_part_1: (46881677, 10)


## Part 2

**Price**

In [22]:
# find the max, min, mean, and std of the item in each store
df_SellPrice0['price_max'] = df_SellPrice0.groupby(['store_id','item_id'])['sell_price'].transform('max')
df_SellPrice0['price_min'] = df_SellPrice0.groupby(['store_id','item_id'])['sell_price'].transform('min')
df_SellPrice0['price_std'] = df_SellPrice0.groupby(['store_id','item_id'])['sell_price'].transform('std')
df_SellPrice0['price_mean'] = df_SellPrice0.groupby(['store_id','item_id'])['sell_price'].transform('mean')

In [23]:
df_SellPrice0['price_norm'] = df_SellPrice0['sell_price']/df_SellPrice0['price_max']

In [24]:
# how many different sell prices for the same item
df_SellPrice0['price_nunique'] = df_SellPrice0.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
# how many different items for the same price
df_SellPrice0['item_nunique'] = df_SellPrice0.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

**price and calendar**

In [25]:
# but would like months and years as "window"
calendar_prices = df_calendar0[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
df_SellPrice0 = df_SellPrice0.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')

In [26]:
# del calendar_prices

In [27]:
# calculate the price "momentum": price over month average and year average
# calculate the average sell_price of the month
# df_SellPrice0.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
df_SellPrice0['price_momentum_m'] = df_SellPrice0['sell_price']/df_SellPrice0.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
df_SellPrice0['price_momentum_y'] = df_SellPrice0['sell_price']/df_SellPrice0.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

# Today's price over yesterday's price
df_SellPrice0['price_momentum'] = df_SellPrice0['sell_price']/df_SellPrice0.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))

In [28]:
# del df_SellPrice0['month'], df_SellPrice0['year']

In [29]:
# merge and save part 2
OriginalCol = list(df_grid)
df_grid = df_grid.merge(df_SellPrice0, on=['store_id','item_id','wm_yr_wk'], how='left')

In [30]:
KeepCol = [col for col in list(df_grid) if col not in OriginalCol]
df_grid = df_grid[MAIN_INDEX+KeepCol]
df_grid = reduce_mem_usage(df_grid)
print('Size of grid_part_2: {}'.format(df_grid.shape))

Memory usage decreased from 2192.27 Mb to 1924.01 Mb (12.2% reduction)
Size of grid_part_2: (46881677, 15)


In [31]:
df_grid.to_pickle(FilePath+'grid_part_2.pkl')
# print('Size of grid_part_2: {}'.format(df_grid.shape))

In [32]:
# del df_SellPrice0

## part 3

In [33]:
# read the df_grid from part 1
# some features from part 2 are not needed
df_grid = pd.read_pickle(FilePath+'grid_part_1.pkl')

In [34]:
df_grid = df_grid[MAIN_INDEX]

**Calendar**

In [35]:
icols = ['date', 'd', 'event_name_1', 'event_type_1', 'event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']

In [36]:
df_grid = df_grid.merge(df_calendar0[icols], on=['d'], how='left')

In [37]:
icols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
for col in icols:
    df_grid[col] = df_grid[col].astype('category')

In [38]:
df_grid['date'] = pd.to_datetime(df_grid['date'])

In [39]:
# add features from date
df_grid['tm_d'] = df_grid['date'].dt.day.astype(np.int8)
df_grid['tm_w'] = df_grid['date'].dt.week.astype(np.int8)
df_grid['tm_m'] = df_grid['date'].dt.month.astype(np.int8)
# find our which year it is
df_grid['tm_y'] = df_grid['date'].dt.year
df_grid['tm_y'] = (df_grid['tm_y'] - df_grid['tm_y'].min()).astype(np.int8)
# Monday=0, Sunday=6
df_grid['tm_dw'] = df_grid['date'].dt.dayofweek.astype(np.int8)

df_grid['tm_wm'] = df_grid['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)
df_grid['tm_w_end'] = (df_grid['tm_dw']>=5).astype(np.int8) # find weekend

In [40]:
# Remove date
# del df_grid['date']

In [41]:
# Safe part 3
df_grid.to_pickle(FilePath+'grid_part_3.pkl')
print('Size of grid_part_3: {}'.format(df_grid.shape))

Size of grid_part_3: (46881677, 17)


In [42]:
# We don't need calendar_df anymore
# del df_calendar0
# del df_grid

In [43]:
basic = pd.read_pickle(FilePath+'grid_part123.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'MainData/grid_part123.pkl'

In [6]:
pd.set_option('display.max_columns',None)

In [7]:
basic.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum_m,price_momentum_y,price_momentum,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_dw,tm_wm,tm_w_end
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,11101,11101,0.459961,0.500000,0.419922,0.019791,0.476318,0.919922,4.0,16,0.968750,0.949707,NaN,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,11101,11101,1.559570,1.769531,1.559570,0.032715,1.764648,0.881348,2.0,9,0.885742,0.896484,NaN,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,11101,11101,3.169922,3.169922,2.970703,0.046173,2.982422,1.000000,2.0,20,1.064453,1.043945,NaN,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,11101,11101,5.980469,6.519531,5.980469,0.115906,6.468750,0.917480,3.0,71,0.922363,0.959473,NaN,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,11101,11101,0.700195,0.720215,0.680176,0.011353,0.707031,0.972168,3.0,16,0.990234,1.001953,NaN,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
